In [11]:
import numpy as np
import pandas as pd
import cv2
import random
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import metrics
from mostCommon import most_common
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import math

In [12]:
my_data = pd.read_csv('../data/Five_Frames_Per_patient.csv')
patients_data=pd.read_csv('../data/videos_data.csv')
data_array = my_data.to_numpy()
patients_array=patients_data["FileName"].to_numpy()
images_path = '../data/Five_Frames_Per_patient/'
no_of_test_patients=math.floor(patients_array.size*0.2) 
images_data = []
models = []

In [13]:
rand_indexes=[]
test_patients=[]
while len(rand_indexes)<no_of_test_patients:
    h=random.randint(0, 121)
    if h not in rand_indexes:
        rand_indexes.append(h)
rand_indexes = sorted(rand_indexes, reverse=True) 
for i in range(len(rand_indexes)):
    test_patients.append(np.take(patients_array,rand_indexes[i]))
    patients_array= np.delete(patients_array, rand_indexes[i])

In [14]:
#I got my patients names now i need to get their frames from my_data

df=my_data
test_data=[]
for i in range(len(test_patients)):
    contain_values = df[df['FileName'].str.contains(test_patients[i])].to_numpy()
    if len(contain_values) == 0:
        print(test_patients[i]+'        Duplicate')
    test_data.append(contain_values)
    indexes = df[df['FileName'].str.contains(test_patients[i])].index
    df.drop(indexes, inplace = True )

Cov-Atlas-Day+2        Duplicate
Cov-Atlas+(45)        Duplicate


In [15]:
X_train=[]
X_test=[]
y_train=[]
y_test=[]

In [16]:
data_array=df.to_numpy()
for i in data_array:
    img = cv2.imread(images_path + i[0] + '.' + i[2], cv2.IMREAD_GRAYSCALE)
    img = cv2.equalizeHist(img)
    img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_AREA)
    img_1d = img.reshape(224 * 224)
    images_data.append(img_1d)
X_train=np.asarray(images_data)
y = np.asarray(my_data['Label'])

In [17]:
patients_images=[]
for patient in test_data:
    for i in patient:
        img = cv2.imread(images_path + i[0] + '.' + i[2], cv2.IMREAD_GRAYSCALE)
        img = cv2.equalizeHist(img)
        img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_AREA)
        img_1d = img.reshape(224 * 224)
        X_test.append(img_1d)
        y_test.append(i[1])

In [18]:
# Now I have xtrain xtest ytrain ytest ready I should increment them in X
print(len(X_train))
print(len(y))
print(len(X_test))
print(len(y_test))

500
500
110
110


In [19]:
r = []
conf_matricies = []
f1_reports = []
f1_micro = []
for i in range(7):
    r.append(random.randint(0, 1000))
print('Random seeds are: '+str(r))

number_of_models = 21
X = np.asarray(images_data)
y = np.asarray(my_data['Label'])

for k in range(len(r)):
    print('--------------------------------- Iteration ' + str(k) + '--------------------------------- \n')
    Xandy = []
    models = []
    # creating models
    for i in range(number_of_models):
        models.append(make_pipeline(StandardScaler(), SVC(gamma='auto', random_state=r[k])))

    # Creating array test splits for models
    # X_train, X_test, y_train, y_test
    for i in range(number_of_models):
        #append x train x test y train ytest to x
        x=[]
        x.append(X_train)
        x.append(X_test)
        x.append(y)
        x.append(y_test)
        Xandy.append(x)

    print('Equalizing Data...')
    for i in range(number_of_models):
        x = Xandy[i][0]
        y1 = Xandy[i][2]
        cov = np.count_nonzero(y1 == 'covid')
        pneu = np.count_nonzero(y1 == 'pneumonia')
        reg = np.count_nonzero(y1 == 'regular')
        # deleting random pneumonia elements from each model set
        while cov != pneu:
            index = random.randint(0, len(x) - 1)
            if y1[index] == 'pneumonia':
                x = np.delete(x, index, 0)
                y1 = np.delete(y1, index, 0)
                pneu -= 1
        # deleting random regular elements from each model set
        while cov != reg:
            index = random.randint(0, len(x) - 1)
            if y1[index] == 'regular':
                x = np.delete(x, index, 0)
                y1 = np.delete(y1, index, 0)
                reg -= 1
        Xandy[i][0] = x
        Xandy[i][2] = y1

    print('Normalizing and applying PCA...')
    for i in range(number_of_models):
        scaler = StandardScaler()
        # Fit on training set only
        scaler.fit(Xandy[i][0])

        # Apply transform on both training and test set
        Xandy[i][0] = scaler.transform(Xandy[i][0])
        Xandy[i][1] = scaler.transform(Xandy[i][1])
    for i in range(number_of_models):
        pca = PCA(n_components=270)
        pca.fit(Xandy[i][0])
        Xandy[i][0] = pca.transform(Xandy[i][0])
        Xandy[i][1] = pca.transform(Xandy[i][1])
    print(len(Xandy[0][0][0]))
    print('Training Models...')
    for i in range(number_of_models):
        models[i].fit(Xandy[i][0], Xandy[i][2])

    print('getting predictions of each model...')
    predicted = []
    for i in range(number_of_models):
        predicted.append(models[i].predict(Xandy[i][1]))

    voted_prediction = []
    for i in range(len(predicted[0])):
        voting = []
        for j in range(len(predicted)):
            voting.append(predicted[j][i])
        voted_prediction.append(most_common(voting))
    conf_matrix = confusion_matrix(Xandy[0][3], voted_prediction)
    print(conf_matrix)
    conf_matricies.append(conf_matrix)

    classification_report = metrics.classification_report(Xandy[0][3], voted_prediction, digits=3)
    print(classification_report)
    f1_reports.append(classification_report)
    f1_micro.append(f1_score(Xandy[0][3], voted_prediction, average='micro'))
    print(f1_score(Xandy[0][3], voted_prediction, average='micro'))

print('Average f1 micro of all iterations is: ' + str(sum(f1_micro) / len(f1_micro)))

print('Saving models....')
np.save('models', models)
print('All Done :)')

Random seeds are: [704, 59, 933, 637, 415, 400, 455]
--------------------------------- Iteration 0--------------------------------- 

Equalizing Data...
Normalizing and applying PCA...
270
Training Models...
getting predictions of each model...
[[18  2  5]
 [ 6 32  2]
 [ 5 10 30]]
              precision    recall  f1-score   support

       covid      0.621     0.720     0.667        25
   pneumonia      0.727     0.800     0.762        40
     regular      0.811     0.667     0.732        45

    accuracy                          0.727       110
   macro avg      0.720     0.729     0.720       110
weighted avg      0.737     0.727     0.728       110

--------------------------------- Iteration 1--------------------------------- 

Equalizing Data...
Normalizing and applying PCA...
270
Training Models...
getting predictions of each model...
[[18  2  5]
 [ 6 32  2]
 [ 5 10 30]]
              precision    recall  f1-score   support

       covid      0.621     0.720     0.667        25

In [ ]:
#count how many patient were identified correctly

In [ ]:
cm=[[0,0,0],
    [0,0,0],
    [0,0,0]]
for i in range(len(patient_prediction)):
    a=patient_prediction[i]
    b=patient_test[i]
    if a == b and a=='covid':
        cm[0][0]=cm[0][0]+1
    if a == b and a=='pneumonia':
        cm[1][1]=cm[1][1]+1
    if a == b and a=='regular':
        cm[2][2]=cm[2][2]+1
    if a=='covid' and b=='pneumonia':
        cm[0][1]=cm[0][1]+1
    if a=='covid' and b=='regular':
        cm[0][2]=cm[0][2]+1
    if a=='pneumonia' and b=='covid':
        cm[1][0]=cm[1][0]+1
    if a=='pneumonia' and b=='regular':
        cm[1][2]=cm[1][2]+1
    if a=='regular' and b=='covid':
        cm[2][0]=cm[2][0]+1
    if a=='regular' and b=='pneumonia':
        cm[2][1]=cm[2][1]+1

In [70]:
print(cm[0])
print(cm[1])
print(cm[2])

[2, 1, 1]
[0, 4, 2]
[0, 0, 12]
